--- 
## Wikitext 2 data loading code

To help you move faster, here is the code to generate the dataloaders for each task

#### Prereqs:
1. Make sure you have 'wiki-news-300d-1M.vec' downloaded locally
2. Make sure to have 'mnli_val.tsv' and 'mnli_train.tsv' locally

In [5]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import pickle
import torch
import torchvision.models as models
import os
from torchvision import transforms
from torchvision.datasets import  MNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import os
import json
import jsonlines
import numpy as np
from collections import defaultdict
from torch import nn
import numpy

In [ ]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    embedding_size = 300
    max_vocab_size = 35000
    embedding_dict = np.random.randn(max_vocab_size+2, embedding_size)
    all_train_tokens = []
    i = 0
    
    for line in fin:
        tokens = line.rstrip().split(' ')
        all_train_tokens.append(tokens[0])
        embedding_dict[i+2] = list(map(float, tokens[1:]))
        i += 1
        if i == max_vocab_size:
            break
            
    return embedding_dict, all_train_tokens
  
# download the vectors yourself
fasttext_embedding_dict, all_fasttext_tokens = load_vectors('wiki-news-300d-1M.vec')
  
class LMDataset(Dataset):
    def __init__(self, list_of_token_lists):
        self.input_tensors = []
        self.target_tensors = []

        for sample in list_of_token_lists:
            self.input_tensors.append(torch.tensor([sample[:-1]], dtype=torch.long))
            self.target_tensors.append(torch.tensor([sample[1:]], dtype=torch.long))

    def __len__(self):
        return len(self.input_tensors)

    def __getitem__(self, idx):
        return (self.input_tensors[idx], self.target_tensors[idx])


def tokenize_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for l in tqdm(dataset):
            l = ['<bos>'] + l + ['<eos>']
            encoded_l = dictionary.encode_token_seq(l)
            _current_dictified.append(encoded_l)
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def tokenize_mnli_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for s1, s2 in tqdm(dataset):
            s1 = ['<bos>'] + s1 + ['<eos>']
            s2 = ['<bos>'] + s2 + ['<eos>']
            encoded_s1 = dictionary.encode_token_seq(s1)            
            encoded_s2 = dictionary.encode_token_seq(s2)
            _current_dictified.append([encoded_s1, encoded_s2])
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = max([t.size(-1) for t in list_of_tensors])
    padded_list = []
    for t in list_of_tensors:
        padded_tensor = torch.cat(
            [t, torch.tensor([[pad_token] * (max_length - t.size(-1))], dtype=torch.long)], dim=-1)
        padded_list.append(padded_tensor)

    padded_tensor = torch.cat(padded_list, dim=0)
    return padded_tensor


def pad_collate_fn(pad_idx, batch):
    input_list = [s[0] for s in batch]
    target_list = [s[1] for s in batch]
    input_tensor = pad_list_of_tensors(input_list, pad_idx)
    target_tensor = pad_list_of_tensors(target_list, pad_idx)
    return input_tensor, target_tensor


def load_wikitext(data_dir):
    import subprocess
    filename = os.path.join(data_dir, 'wikitext2-sentencized.json')
    if not os.path.exists(filename):
        os.makedirs(data_dir, exist_ok=True)
        url = "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json"
        args = ['wget', '-O', filename, url]
        subprocess.call(args)
    raw_datasets = json.load(open(filename, 'r'))
    for name in raw_datasets:
        raw_datasets[name] = [x.split() for x in raw_datasets[name]]

    if os.path.exists(os.path.join(data_dir, 'vocab.pkl')):
        vocab = pickle.load(open(os.path.join(data_dir, 'vocab.pkl'), 'rb'))
    else:
        vocab = Dictionary(raw_datasets, include_valid=False)
        pickle.dump(vocab, open(os.path.join(data_dir, 'vocab.pkl'), 'wb'))

    tokenized_datasets = tokenize_dataset(raw_datasets, vocab)
    datasets = {name: LMDataset(ds) for name, ds in tokenized_datasets.items()}
    print("Vocab size: %d" % (len(vocab)))
    return raw_datasets, datasets, vocab


class Dictionary(object):
    def __init__(self, datasets, include_valid=False):
        self.tokens = []
        self.ids = {}
        self.counts = {}
        
        # add special tokens
        self.add_token('<bos>')
        self.add_token('<eos>')
        self.add_token('<pad>')
        self.add_token('<unk>')
        
        for line in tqdm(datasets['train']):
            for w in line:
                self.add_token(w)
                    
        if include_valid is True:
            for line in tqdm(datasets['valid']):
                for w in line:
                    self.add_token(w)
                            
    def add_token(self, w):
        if w not in self.tokens:
            self.tokens.append(w)
            _w_id = len(self.tokens) - 1
            self.ids[w] = _w_id
            self.counts[w] = 1
        else:
            self.counts[w] += 1

    def get_id(self, w):
        return self.ids[w]
    
    def get_token(self, idx):
        return self.tokens[idx]
    
    def decode_idx_seq(self, l):
        return [self.tokens[i] for i in l]
    
    def encode_token_seq(self, l):
        return [self.ids[i] if i in self.ids else self.ids['<unk>'] for i in l]
    
    def __len__(self):
        return len(self.tokens)
 

raw_datasets, datasets, vocab = load_wikitext(os.getcwd())

data_loaders = {name: DataLoader(datasets[name], batch_size=32, shuffle=True,
                                     collate_fn=lambda x: pad_collate_fn(vocab.get_id('<pad>'), x))
                    for name in datasets}
wk2_train_dataloader = data_loaders['train']
wk2_val_dataloader = data_loaders['valid']
wk2_test_dataloader = data_loaders['test']

In [ ]:
print('WIKITEXT 2 dataset statistics')
print(f'training samples:{len(wk2_train_dataloader)}')
print(f'val samples:{len(wk2_val_dataloader)}')
print(f'test samples:{len(wk2_test_dataloader)}')

---  
## MNLI loading code

In [1]:
y_label_map = {'contradiction':0,'neutral':1,'entailment':2}
import pandas as pd
import io
from collections import Counter

def get_string_tokenized_data(data):
    
    tokenized_data_x = [];
    y_labels = []
    all_tokens = [];
    
    for i,x in enumerate(data):
        label = x[2]
        if label == 'nan':
            continue
        
        label = y_label_map[label]
        y_labels.append(label)
        
        dp = [x[0].split(), x[1].split()]
        tokenized_data_x.append(dp)
        all_tokens += (dp[0] + dp[1])
        

    return all_tokens, tokenized_data_x, y_labels
        


# convert token to id in the dataset
def token2index_dataset(tokens_data, token2id):
    indices_data = []
    for tokens1, tokens2 in tokens_data:
        index_list1 = [token2id[token] if token in token2id else UNK_IDX for token in tokens1]
        index_list2 = [token2id[token] if token in token2id else UNK_IDX for token in tokens2]
        indices_data.append([index_list1, index_list2])
    return indices_data
  
# convert token to id in the dataset
def token2index_using_wikitext2_dict(tokens_data, vocab):
    indices_data = []
    for tokens1, tokens2 in tokens_data:
        index_list1 = vocab.encode_token_seq(tokens1)
        index_list2 = vocab.encode_token_seq(tokens2)
        indices_data.append([index_list1, index_list2])
    return indices_data


def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(MAX_VOCAB_SIZE))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

--- 
#### Validation set (used as test set)

In [ ]:
# LOAD VAL

val_df = pd.read_csv('mnli_val.tsv', sep="\t")
print(val_df.head(2))

val_df  = np.array(val_df)
val_df = val_df.astype(str)
val_genre_list = val_df[:, 3]

In [ ]:
_, val_data_x, val_data_y = get_string_tokenized_data(val_df)
del val_df

---   
#### Training set (used as training, validation)

In [ ]:
train_df = pd.read_csv('mnli_train.tsv', sep="\t")

train_df  = np.array(train_df)
train_df = train_df.astype(str)
train_genre_list = train_df[:, 3]

In [ ]:
_, train_data_x, train_data_y = get_string_tokenized_data(train_df)
del train_df

In [ ]:
# token2id, id2token = build_vocab(all_fasttext_tokens)
# train_data_indices = token2index_using_wikitext2_dict(train_data_x, vocab)
# val_data_indices = token2index_using_wikitext2_dict(val_data_x, vocab)


mnli_raw_datasets = {'train': train_data_x, 'val': val_data_x}
mnli_tokenized_datasets = tokenize_mnli_dataset(mnli_raw_datasets, vocab)

train_data_indices = mnli_tokenized_datasets['train']
val_data_indices = mnli_tokenized_datasets['val']


# double checking
print('\n')
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))

# del train_data_x
# del train_data_y
# del val_data_x
# del val_data_y
del mnli_tokenized_datasets

In [ ]:
unique_genre = list(set(val_genre_list));
nb_classes = len(y_label_map)

In [ ]:
MAX_SENTENCE_LENGTH = 200
class MNLIDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_x, target_list):
        """
        @param data_list: list of newsgroup tokens
        @param target_list: list of newsgroup targets

        """
        self.data_x = data_x;
        self.target_list = target_list
        
        assert(len(data_x) == len(target_list))

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        prem_token_idx = self.data_x[key][0][:MAX_SENTENCE_LENGTH]
        hyp_token_idx = self.data_x[key][1][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [prem_token_idx, hyp_token_idx, label]


def encode_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    prem_data_list = []
    hyp_data_list = []
    label_list = []
    length_list = []
    # print("collate batch: ", batch[0][0])
    # batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
    # padding
    for datum in batch:
        prem_padded_vec = np.pad(np.array(datum[0]),
                                 pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[0]))),
                                 mode="constant", constant_values=0)
        hyp_padded_vec = np.pad(np.array(datum[1]),
                                pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[1]))),
                                mode="constant", constant_values=0)
        prem_data_list.append(prem_padded_vec)
        hyp_data_list.append(hyp_padded_vec)
    return [torch.from_numpy((np.array(prem_data_list))), torch.from_numpy(np.array(hyp_data_list)),
            torch.LongTensor(label_list)]

In [ ]:
BATCH_SIZE = 32
nb_train_samples = int(0.95 * len(train_data_indices))
nb_val_samples = len(train_data_indices) - nb_train_samples

# train/val split
train_val_dataset = MNLIDataset(train_data_indices, train_data_y)
train_dataset, val_dataset = random_split(train_val_dataset, [nb_train_samples, nb_val_samples])

# train loader
train_mnli_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

# val loader
val_mnli_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

# test loader
test_dataset = MNLIDataset(val_data_indices, val_data_y)
test_mnli_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

In [ ]:
print('MNLI dataset statistics')
print(f'training samples:{len(train_mnli_loader)}')
print(f'val samples:{len(val_mnli_loader)}')
print(f'test samples:{len(test_mnli_loader)}')